## 字典格式

In [76]:
## 字典格式
article = {
    'comp':'Gossip',
    'category':'問卦',
    'url':'',
    'author':'',
    'title':'',
    'date':'',
    'push_nums':'',
    'main_content':'',
    'push_content':[  #推文會有多個，所以用list裝
        {
            'tag':'',
            'userid':'',
            'datetime':'',
            'content':'',
        }
    ],
    'push_count':{  #算數量的只會有一個，用字典裝就好
            'all':'',
            'good':'',
            'bad':'',
            'neutral':'',
            'my_push_nums':'good-bad',
    }
    
}

## 抓日期

In [78]:
import datetime
#今天日期 
today = datetime.datetime.now()
fileName = today.strftime('%Y%m%d')

## Step1 讀取Url File 爬取內文 

In [74]:
#coding=utf-8 
import requests
from bs4 import BeautifulSoup

#通過18禁頁面
playload = {
    'from':'/bbs/Gossiping/index.html',
    'yes':'yes'
}
rs = requests.session() #設定一個session
res1 = rs.post('https://www.ptt.cc/ask/over18',verify=False,data = playload) #透過session記錄通過18禁頁面

#基本設定
a = 0 #限制跑幾圈
PttName="Gossiping"
all_json_str = [] #存所有資訊的 json

#開啟 url檔案
with open('C:/Users/ytchen/Desktop/ptt/{}.txt'.format(fileName),'r') as f:
    for line in f.readlines():
        tempList = line.split(',')
        
        #Get Information
        url = 'http'+tempList[0].split('http')[1] #截取下來的網址，http前面會有奇怪的東西，要刪掉
        category = tempList[1]
        push_nums = tempList[2]
        title = tempList[3]
        date = tempList[4]
        author = tempList[5]      
#         print line
#         print url,category,push_nums,title,date,auth

        #開始爬取文章
        res = rs.get(url,verify=False)
        soup = BeautifulSoup(res.text)
        
        #mydate
        mydate = soup.select('.article-meta-value')[3].text
        
        #content
        content = soup.find(id="main-content").text
        target_content=u'※ 發信站: 批踢踢實業坊(ptt.cc),'
        content = content.split(target_content)
        content = content[0].split(mydate)
        main_content = content[1].replace('\n', ' ').replace('\t', ' ')
        
        # message
        all_push_content = []
        num , good , bad , neutral = 0,0,0,0
        
        for tag in soup.select('div.push'):
            num += 1
            push_tag = tag.find("span", {'class': 'push-tag'}).text
            push_userid = tag.find("span", {'class': 'push-userid'}).text      
            push_content = tag.find("span", {'class': 'push-content'}).text.replace('\n', '').replace('\t', '').strip()
            push_content = push_content[1:].strip()
#             print "push_content:",push_content
            push_ipdatetime = tag.find("span", {'class': 'push-ipdatetime'}).text.replace('\n', '').replace('\t', '').strip()

            message = {
                'tag':push_tag,
                'userid':push_userid,
                'content':push_content,
                'datetime':push_ipdatetime,
            }

            #存取所有推文
            all_push_content.append(message)
        
            #計算推文數量
            if push_tag == u'推 ':
                good += 1
            elif push_tag == u'噓 ':
                bad += 1
            else:
                neutral += 1
        
        push_count = {
            'all':num,
            'good':good,
            'bad':bad,
            'neutral':neutral,
            'my_push_nums':good-bad,
        }
        
#         print all_push_content
#         print push_count
        
        #存取每一篇資訊
        ptt_news = {
            'comp':PttName,
            'category':category,
            'url':url,
            'author':author,
            'title':title,
            'date':date,
            'push_nums':push_nums,
            'main_content':main_content,
            'push_content':all_push_content,
            'push_count':push_count,
        }
        all_json_str.append(ptt_news)
        


        
        #限制迴圈次數
        a += 1
        if a >= 1 :
            break
            
# print all_json_str[0]
# print all_json_str[0]['title']
# print all_json_str[0]['push_nums']
# print all_json_str[0]['main_content']
# print all_json_str[0]['url']
# print all_json_str[0]['push_content'][5]['content']
# print all_json_str[0]['push_count']['my_push_nums']
# print all_json_str[0]['push_count']['all']

{'category': '\xe6\x96\xb0\xe8\x81\x9e', 'date': '20160826', 'push_count': {'my_push_nums': 3, 'all': 13, 'good': 3, 'neutral': 10, 'bad': 0}, 'push_content': [{'content': u'\u9019\u5169\u4f4d\u4e94\u6a13\u90fd\u53ef\u4ee5', 'tag': u'\u63a8 ', 'userid': u'GaryMatthews', 'datetime': u'08/26 23:51'}, {'content': u'KMT\u4e0d\u610f\u2026\u54a6\uff1f', 'tag': u'\u2192 ', 'userid': u'james732', 'datetime': u'08/26 23:51'}, {'content': u'\u8463\u4e8b\u9577\u611b\u6253\u9ad8\u723e\u592b \u7e3d\u7d93\u7406\u8d77\u78bc\u4e5f\u6703\u6709\u4e00\u5957\u7403\u5177 \u4e0d\u5947\u602a', 'tag': u'\u2192 ', 'userid': u'shippai', 'datetime': u'08/26 23:52'}, {'content': u'\u5e6b\u6905\u5b50 QQ', 'tag': u'\u2192 ', 'userid': u'TheDragonBug', 'datetime': u'08/26 23:52'}, {'content': u'\u770b\u4f86\u6c34\u725b\u4f2f\u9000\u4e86\u6bd4\u8f03\u6709\u6642\u9593...', 'tag': u'\u2192 ', 'userid': u'joumay', 'datetime': u'08/26 23:52'}, {'content': u'14\u7d1a\u6709\u9ede\u5f31', 'tag': u'\u2192 ', 'userid': u'rial

In [34]:
#coding=utf-8 
import requests
from bs4 import BeautifulSoup

playload = {
    'from':'/bbs/Gossiping/index.html',
    'yes':'yes'
}

#設定一個session
rs = requests.session()

#透過session記錄通過18禁頁面
res1 = rs.post('https://www.ptt.cc/ask/over18',verify=False,data = playload)

#進入文章開始爬取

url = 'https://www.ptt.cc/bbs/Gossiping/M.1473225217.A.040.html'
res = rs.get(url,verify=False)
soup = BeautifulSoup(res.text)


#date
mydate = soup.select('.article-meta-value')[3].text

# content
content = soup.find(id="main-content").text
target_content=u'※ 發信站: 批踢踢實業坊(ptt.cc),'
content = content.split(target_content)
content = content[0].split(mydate)
main_content = content[1].replace('\n', ' ').replace('\t', ' ')
print 'content:',main_content

# message
all_push_content = []
num , good , bad , normal = 0,0,0,0

for tag in soup.select('div.push'):
    num += 1
    push_tag = tag.find("span", {'class': 'push-tag'}).text
#     print "push_tag:",push_tag
    push_userid = tag.find("span", {'class': 'push-userid'}).text      
#     print "push_userid:",push_userid
    push_content = tag.find("span", {'class': 'push-content'}).text.replace('\n', '').replace('\t', '').strip()
    push_content = push_content[1:].strip()
    print "push_content:",push_content
    push_ipdatetime = tag.find("span", {'class': 'push-ipdatetime'}).text.replace('\n', '').replace('\t', '').strip()
#     print "push-ipdatetime:",push_ipdatetime
    
    message = {
        'push_tag':push_tag,
        'push_userid':push_userid,
        'push_content':push_content,
        'push_datetime':push_ipdatetime,
    }
    
    all_push_content.append(message)

    
    if push_tag == u'推 ':
        good += 1
    elif push_tag == u'噓 ':
        bad += 1
    else:
        normal += 1
push_count = {
    'all':num,
    'good':good,
    'bad':bad,
    'normal':normal,
    'push_nums':good-bad,
}

print all_push_content
print push_count

content:  先說 30多歲老妹 我不行  幹基掰吵死惹啦  帶耳機在那邊看韓劇ㄘㄘ笑  都不用午休喔幹  請問有什麼方法讓這種人閉嘴  有瓜?  謝謝各餵 ※ 八卦板務請到 GossipPicket 檢舉板詢問 ※ a.張貼問卦請注意，充實文章內容、是否有專板，本板並非萬能問板。 ※ b.一天只能張貼 "兩則" 問卦，自刪及被刪也算兩篇之內， ※   超貼者將被水桶，請注意! ※ c.本看板嚴格禁止政治問卦，發文問卦前請先仔細閱讀相關板規。 ※ d.一行文、未滿20字視同鬧板，請注意！ ※ (↑看完提醒請刪除ctrl + y)  -- 
push_content: 阿拉巴瓜
push_content: 月
push_content: 沒肛 我讓五樓肛喇
push_content: 我討厭那種吃飯超大聲的 噁心
push_content: 每次聽到都超想吐  真的超噁 吃飯吃那麼大聲幹嘛
push_content: 喔我光是想到就好想吐  幹你娘
push_content: 我還沒吃飯肚子好餓辣
push_content: 懶趴塞他嘴你可以的
push_content: 走過去跟她說 這部我也很喜歡ㄛ  明天開始她就不會看了
push_content: 樓上高招 讚
push_content: 三杯
push_content: 直接跟她講請她小聲點  你們該不會沒交流過吧?
push_content: 肛她
push_content: 122
push_content: 走過去說你喜歡她這樣ㄘㄘ洨，他明天就離職惹
[{'push_datetime': u'09/07 13:13', 'push_userid': u'bradpete', 'push_tag': u'\u2192 ', 'push_content': u'\u963f\u62c9\u5df4\u74dc'}, {'push_datetime': u'09/07 13:13', 'push_userid': u'Miyanishi25', 'push_tag': u'\u63a8 ', 'push_content': u'\u6708'}, {'push_datetime': u'09/07 13:14', 'push_userid': u'shevchen', 'push_tag': u'\u2

## 神人寫好的爬蟲
- https://github.com/wy36101299/PTTcrawler

In [ ]:
#coding=utf-8 
import re
import sys
import json
import requests
import io
import random
from time import sleep
from datetime import datetime
from bs4 import BeautifulSoup  
requests.packages.urllib3.disable_warnings()

PttName="Gossiping"
load={
'from':'/bbs/'+PttName+'/index.html',
'yes':'yes' 
}

rs=requests.session()
res=rs.post('https://www.ptt.cc/ask/over18',verify=False,data=load)
FILENAME=""

def PageCount(PttName):
    res=rs.get('https://www.ptt.cc/bbs/'+PttName+'/index.html',verify=False)
    soup=BeautifulSoup(res.text,'html.parser')
    ALLpageURL = soup.select('.btn.wide')[1]['href']
    ALLpage=int(getPageNumber(ALLpageURL))+1
    return  ALLpage 

def crawler(PttName,ParsingPage):
        ALLpage=PageCount(PttName)
        g_id = 0;
 
	for number  in range(ALLpage, ALLpage-int(ParsingPage),-1):
            res=rs.get('https://www.ptt.cc/bbs/'+PttName+'/index'+str(number)+'.html',verify=False)
            soup = BeautifulSoup(res.text,'html.parser')
	    for tag in soup.select('div.title'):
		try:
                    atag=tag.find('a')
                    time=random.uniform(0, 1)/5
                    #print 'time:',time
                    sleep(time)
                    if(atag):
                       URL=atag['href']   
                       link='https://www.ptt.cc'+URL
                       #print link
	               g_id = g_id+1
		       parseGos(link,g_id)                     
		except:
                    print 'error:',URL
 
def parseGos(link , g_id):
        res=rs.get(link,verify=False)
        soup = BeautifulSoup(res.text,'html.parser')
        # author
        author  = soup.select('.article-meta-value')[0].text
        #author = soup.find("span", {'class': 'article-meta-value'}).text              
        #print 'author:',author
        # title
        title = soup.select('.article-meta-value')[2].text
        #print 'title:',title
        # date
        date = soup.select('.article-meta-value')[3].text
        #print 'date:',date
        # ip       
        try:
                targetIP=u'※ 發信站: 批踢踢實業坊'
                ip =  soup.find(string = re.compile(targetIP))
                ip = re.search(r"[0-9]*\.[0-9]*\.[0-9]*\.[0-9]*",ip).group()
        except:
                ip = "ip is not find"
        #print 'ip:',ip

        # content
        content = soup.find(id="main-content").text
        target_content=u'※ 發信站: 批踢踢實業坊(ptt.cc),'
        content = content.split(target_content)
        content = content[0].split(date)
        main_content = content[1].replace('\n', '  ').replace('\t', '  ')
        #print 'content:',main_content
        
        # message
        num , g , b , n ,message = 0,0,0,0,{}
        for tag in soup.select('div.push'):
                num += 1
                push_tag = tag.find("span", {'class': 'push-tag'}).text
                #print "push_tag:",push_tag
                push_userid = tag.find("span", {'class': 'push-userid'}).text       
                #print "push_userid:",push_userid
                push_content = tag.find("span", {'class': 'push-content'}).text   
                push_content = push_content[1:]
                #print "push_content:",push_content
                push_ipdatetime = tag.find("span", {'class': 'push-ipdatetime'}).text   
                push_ipdatetime = remove(push_ipdatetime, '\n')
                #print "push-ipdatetime:",push_ipdatetime 
                
                message[num]={"狀態":push_tag.encode('utf-8'),"留言者":push_userid.encode('utf-8'),
                              "留言內容":push_content.encode('utf-8'),"留言時間":push_ipdatetime.encode('utf-8')}
                if push_tag == u'推 ':
                        g += 1
                elif push_tag == u'噓 ':
                        b += 1
                else:
                        n += 1
  
        messageNum = {"g":g,"b":b,"n":n,"all":num}
        # json-data  type(d) dict
          
        d={ "a_ID":g_id , "b_作者":author.encode('utf-8'), "c_標題":title.encode('utf-8'), "d_日期":date.encode('utf-8'),
            "e_ip":ip.encode('utf-8'), "f_內文":main_content.encode('utf-8'), "g_推文":message,"h_推文總數":messageNum}
        json_data = json.dumps(d,ensure_ascii=False,indent=4,sort_keys=True)+','
	store(json_data) 

def store(data):
    with open(FILENAME, 'a') as f:
        f.write(data)
        
def remove(value, deletechars):
    for c in deletechars:
        value = value.replace(c,'')
    return value.rstrip();
   

def getPageNumber(content) :
    startIndex = content.find('index')
    endIndex = content.find('.html')
    pageNumber = content[startIndex+5 : endIndex]
    return pageNumber

if __name__ == "__main__":  
   PttName = str(sys.argv[1])
   ParsingPage = int(sys.argv[2])
   FILENAME='data-'+PttName+'-'+datetime.now().strftime('%Y-%m-%d-%H-%M-%S')+'.json'
   store('[') 
   print 'Start parsing [',PttName,']....'
   crawler(PttName,ParsingPage)
   store(']') 
   

   with open(FILENAME, 'r') as f:
        p = f.read()
   with open(FILENAME, 'w') as f:
        #f.write(p.replace(',]',']'))
        f.write(p[:-2]+']')   